# Learning Objectives

By the end of this lab, you will have

- Implemented Convolution, ReLU, and Max-Pooling layers
- Created a reusable convolutional block layer
- Verfied the correctness of your implementations with gradient checking

Let's get started!

# Layer Interface

Recall when implementing a layer to make it conform to the following interface.

In [1]:
class Layer:
    def forward(self, inputs):
        raise NotImplementedError('Forward pass not implemented!')
        
    def backward(self, dout):
        raise NotImplementedError('Backward pass not implemented!')

# Max-Pooling Layer

Consider a 1D Max-Pooling layer described by the computational graph

![Simple Max Pooling Layer](images/Simple%20Max%20Pooling%20Layer.png)
where

$$
z = \max(\mathbf{h}).
$$

### Questions

- How many dimensions in $\nabla_h$ will be non-zero assuming there is a unique $h_i$ such that $h_i = \max(\mathbf{h})$?
- What if there are two values $h_i$ and $h_j$ such that $h_i = h_j = \max(\mathbf{h})$?

In [ ]:
# max pooling = max( 4x4 region)
# for a one dimensional array, there will only be one max. Therefore, there will be N-1 dimensions that are zero

In [3]:
import numpy as np

In [4]:
h = np.array([1,2,3,4,5,2])

In [5]:
h

array([1, 2, 3, 4, 5, 2])

In [17]:
class Max_Pool(Layer):
    def forward(self,h):
        self.h = h
        z = max(h)
        
        self.z = z if z > 0 else 0
        return z
    def backward(self,incoming_grad ):
        dz = incoming_grad
        if self.z >0:
            dh = 1
        else:
            dh = 0
        return dh * dz

In [18]:
max_pool = Max_Pool()

In [19]:
max_pool.forward(h)

5

In [20]:
max_pool.backward(1)

1

### Tasks

- Implement a 1D Max-Pooling layer

# ReLU Layer

Consider the ReLU layer described by the computational graph

![Simple ReLU Layer](images/Simple%20ReLU%20Layer.png)
where

$$
\mathbf{h}_{i} = 
\begin{cases} 
0 & \text{if } \mathbf{a}_{i} \leq 0 \\
\mathbf{a}_{i} & \text{otherwise}
\end{cases}.
$$

### Questions

- What will $\nabla_a$ be if $a_i < 0$ for all $i$?
- What if $a_i > 0$ for all $i$?

### Tasks

- Implement a 1D ReLU layer

# Convolutional Layer

Consider a 1D convolution layer with a single filter $w$ described by the computational graph

![Simple Conv1D Layer](images/Simple%20Conv1D%20Layer.png)
where $a_i = w * x_i$. Note since $w \in \mathbb{R}$, we are performing a 1x1 convolution. Further assume that we are only dealing with a stride of 1.

### Questions

- How many elements in $\mathbf{a}$ does $x_i$ influence?
- How many elements in $\mathbf{a}$ does $w$ influence?

In [ ]:
# How many elements in  a  does  xi  influence?
## Xi will only influence one element in a

### Tasks

- Implement a 1D convolutional layer

In [34]:
class Convolution_Layer_scalar(Layer):
    def forward(self,x,w):
        self.x = x
        self.w=w
        convolution = [i*w for i in x]
        return convolution
    def backward(self,incoming_grad):
        da = incoming_grad
        dw = self.x * da
        dx = self.w * da
        return dw,dx

In [35]:
x = np.array([1,4,2,5,3])

In [36]:
w=10

In [37]:
convolution_layer = Convolution_Layer_scalar()

In [38]:
convolution_layer.forward(x,w)

[10, 40, 20, 50, 30]

In [39]:
convolution_layer.backward(1)

(array([1, 4, 2, 5, 3]), 10)


{\displaystyle {\begin{aligned}(f*g_{N})[n]&=\sum _{m=0}^{N-1}f[m]\ g_{N}[n-m]\\&=\sum _{m=0}^{n}f[m]\ g[n-m]+\sum _{m\,=\,n+1}^{N-1}f[m]\ g[N+n-m]\\&=\sum _{m=0}^{N-1}f[m]\ g[(n-m)_{\bmod {N}}]\equiv (f*_{N}g)[n]\end{aligned}}}

In [42]:
t = np.array([])

In [44]:
t.

IndexError: index 0 is out of bounds for axis 0 with size 0

In [97]:
class Convolution_Layer_vectors(Layer):
    def forward(self,x,w,convolution_padding=1):
        self.x = x
        self.w=w
        convolution = np.zeros(len(self.x)+convolution_padding)

        for y_number in range(len(x)+convolution_padding):
            
            convolution_sum = 0
            for x_idx in range(y_number+1):
                print(x_idx,'x_idx')
                print(y_number-x_idx,"y_number-x_idx")
                try:
                    convolution_sum += x[x_idx]*w[y_number-x_idx]
                except IndexError: ## no weights left
                    pass
            print("convolution sum is {}".format(convolution_sum))
            convolution[y_number]=convolution_sum
        self.convolution = convolution
        return convolution
    def backward(self,incoming_grad):
        pass # Full derivation available online
        

In [91]:
x = np.array([3,4,5])
w = np.array([2,1])

In [92]:
con_lay_vec = Convolution_Layer_vectors()

In [96]:
con_lay_vec.forward(x,w,1)

0 x_idx
0 y_number-x_idx
convolution sum is 6
0 x_idx
1 y_number-x_idx
1 x_idx
0 y_number-x_idx
convolution sum is 11
0 x_idx
2 y_number-x_idx
1 x_idx
1 y_number-x_idx
2 x_idx
0 y_number-x_idx
convolution sum is 14
0 x_idx
3 y_number-x_idx
1 x_idx
2 y_number-x_idx
2 x_idx
1 y_number-x_idx
3 x_idx
0 y_number-x_idx
convolution sum is 5


array([  6.,  11.,  14.,   5.])

### Convolutional Block

- Consider a convolutional block layer described by the computational graph

$$
\underset{w \in \mathbb{R}}{\overset{\mathbf{x} \in \mathbb{R}^N}{\longrightarrow}}
\text{Conv}
\longrightarrow
\text{ReLU}
\longrightarrow
\text{Max Pool}
\overset{h \in \mathbb{R}}{\longrightarrow}
$$

### Tasks

- Implement a convolutional block layer in terms of Convolutional, ReLU, and Max Pool layers

## Check Your Implementation

An indispensible tool to check your backpropagation code is *gradient checking*. Gradient checking works by

1. Running your backward pass to compute the gradients
2. Approximating the gradients with finite differences
3. Compares these two values and returns success if they are close and fails otherwise

### Tasks

- Run the following code cell to gradient check your convolutional block

### Explanation

- The code will create a vector $x$ of five random numbers and a random filter $w$. It approxiates $\nabla{x}$ and $\nabla{w}$ and compares those values against the values of $\nabla{w}$ and $\nabla{x}$ your `ConvBlock.backward()` method returns.

## Bonus Activities

- Implement a 1D convolution layer with support for multiple scalar filters
- Implement a 1D convolution layer with one filter which is a vector
- Implement a 1D convolution layer with a set of filters which are vectors
- Implement a 2D convolution layer
- Implement a max-pooling layer which supports local maxes
- Implement a 2D max-pooling layer
- Generalize your code to support minibatches
- Implement a simple trainer class with an SGD optimizer and optimize a CNN on MNIST